In [ ]:
# Make a system which tells whether the person will be
#save from sinking. What factors were
#most likely lead to success-socio-economic
#status, age, gender and more.

In [17]:
#necessary libraries:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

In [4]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

In [5]:
df=sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [6]:
df.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True


In [7]:
df.describe

<bound method NDFrame.describe of      survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult

In [8]:
# Drop unnecessary columns
df = df.drop(['alive', 'who', 'adult_male', 'class', 'embark_town', 'deck', 'embarked'], axis=1)

In [9]:
# Handling missing values
df['age'] = df['age'].fillna(df['age'].mean())

In [10]:
# Convert categorical variables to numerical using one-hot encoding
df = pd.get_dummies(df, columns=['sex', 'alone'])

In [11]:
#Prepare the dataset for LSTM training:

In [12]:
# Split the dataset into features (X) and target (y)
X = df.drop('survived', axis=1)
y = df['survived']


In [14]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [15]:
# Reshape the features for LSTM input shape
X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [18]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train_scaled.shape[1]), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [19]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Train the model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)

Epoch 1/10
23/23 [==============================] - 2s 3ms/step - loss: 0.6785 - accuracy: 0.6966
Epoch 2/10
23/23 [==============================] - 0s 3ms/step - loss: 0.6340 - accuracy: 0.7739
Epoch 3/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5868 - accuracy: 0.7837
Epoch 4/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5381 - accuracy: 0.7907
Epoch 5/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4966 - accuracy: 0.7907
Epoch 6/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4700 - accuracy: 0.7907
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4547 - accuracy: 0.7949
Epoch 8/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4455 - accuracy: 0.8062
Epoch 9/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4392 - accuracy: 0.8104
Epoch 10/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4344 - accuracy: 0.8118


In [22]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_reshaped, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


6/6 [==============================] - 0s 3ms/step - loss: 0.4286 - accuracy: 0.7933
Test Loss: 0.4285818338394165
Test Accuracy: 0.7932960987091064


In [26]:
# Make predictions on new data
new_data = X.sample(5, random_state=42)
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = np.reshape(new_data_scaled, (new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))
predictions = model.predict(new_data_reshaped)

for i, prediction in enumerate(predictions):
    print("Prediction for sample", i+1, ": Survived" if prediction[0] == 1 else "Did not survive")

1/1 [==============================] - 0s 196ms/step
Prediction for sample 1 Did not survive
Prediction for sample 2 Did not survive
Prediction for sample 3 Did not survive
Prediction for sample 4 Did not survive
Prediction for sample 5 Did not survive


In [ ]:
#thankyou😊😊